# 特征工程

In [1]:
import pandas as pd
import datetime
import logging
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler, Normalizer
from sklearn.feature_selection import SelectKBest,chi2
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

from mlxtend.preprocessing import DenseTransformer
from mlxtend.feature_selection import ColumnSelector

In [2]:
def groupby2df(t, name):
    t = t.to_frame()
    t.columns = [name]
    return t

In [3]:
logger = logging.getLogger('ai')
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s  %(filename)s : %(levelname)s  %(message)s')

In [4]:
offline_df = pd.read_csv('../source/ccf_offline_stage1_train.csv', parse_dates = ['Date_received', 'Date'])
online_df = pd.read_csv('../source/ccf_online_stage1_train.csv', parse_dates = ['Date_received', 'Date'])

In [5]:
pred_df = pd.read_csv('../source/ccf_offline_stage1_test_revised.csv', parse_dates = ['Date_received'])

## 数据集划分

In [6]:
time_range = [datetime.datetime(2016, 3, 16), datetime.datetime(2016, 6, 30)]

time_range_date_received = time_range
time_range_date = time_range

FILENAME = 'dataset_pred'

IS_PRED = True

In [7]:
# time_range = [datetime.datetime(2016, 5, 16), datetime.datetime(2016, 6, 15)]

# time_range_date_received = [datetime.datetime(2016, 2, 1), datetime.datetime(2016, 4, 30)]
# time_range_date = [datetime.datetime(2016, 2, 1), datetime.datetime(2016, 5, 15)]

# FILENAME = 'dataset_beta'

# IS_PRED = False

In [8]:
# time_range = [datetime.datetime(2016, 4, 16), datetime.datetime(2016, 5, 15)]

# time_range_date_received = [datetime.datetime(2016, 1, 1), datetime.datetime(2016, 3, 31)]
# time_range_date = [datetime.datetime(2016, 1, 1), datetime.datetime(2016, 4, 15)]

# FILENAME = 'dataset_alpha'

# IS_PRED = False

In [9]:
def _split(row, time_range_date_received, time_range_date):
    if ((row.Date >= time_range_date[0]) & (row.Date <= time_range_date[1])) | ((row.Coupon_id == 0) & (row.Date_received >= time_range_date_received[0]) & (row.Date_received <= time_range_date_received[1])):
        return row
    
def dataset_fetch(time_range):
    dataset = offline_df[(offline_df.Date_received >= time_range[0]) & (offline_df.Date_received <= time_range[1])].copy()
    return dataset
    
def dataset_split(time_range_date_received, time_range_date):
    feature_offline = offline_df.loc[
        ((offline_df.Date >= time_range_date[0]) & (offline_df.Date <= time_range_date[1])) | 
        ((offline_df.Coupon_id == 0) & (offline_df.Date_received >= time_range_date_received[0]) & (offline_df.Date_received <= time_range_date_received[1]))]
    feature_online = online_df.loc[
        ((online_df.Date >= time_range_date[0]) & (online_df.Date <= time_range_date[1])) | 
        ((online_df.Coupon_id == 0) & (online_df.Date_received >= time_range_date_received[0]) & (online_df.Date_received <= time_range_date_received[1]))]
    
    return feature_offline, feature_online

if IS_PRED:
    dataset = pred_df
else:
    dataset = dataset_fetch(time_range)

feature_alpha_offline, feature_alpha_online = dataset_split(time_range_date_received, time_range_date)

## 基础数据特征

### 基础特征抽取

In [10]:
discount_cat = offline_df['Discount_rate'].unique()

In [11]:
def cal_coupon_feature(row):
    if isinstance(row.Discount_rate, str) and row.Discount_rate == 'fixed':
        row.Coupon_type = 2
        return row
    
    if isinstance(row.Discount_rate, float):
        row.Discount = row.Discount_rate
        i, = np.where(discount_cat == row.Discount_rate)
        if len(i)>0:
            row.Coupon_category = i[0]
        return row
    
    arr = row.Discount_rate.split(':')
    if len(arr) == 2:
        row.Discount =  (float(arr[0]) - float(arr[1])) / float(arr[0])
        row.Coupon_type = 1
        row.Base_consume = float(arr[0])
        row.Discount_money = float(arr[1])
    else:
        row.Discount = float(row.Discount_rate)
        
    i, = np.where(discount_cat == row.Discount_rate)
    if len(i)>0:
        row.Coupon_category = i[0]
    
    return row

In [12]:
def cal_previous_duration(row):
    if row['User_id'] == row['Previous_user_id'] and row['Date_received'] is not None and row['Previous_date_received'] is not None:
        return (row.Date_received - row.Previous_date_received).days
    
    return 0

def cal_next_duration(row):
    if row['User_id'] == row['Next_user_id'] and row['Date_received'] is not None and row['Next_date_received'] is not None:
        return (row.Next_date_received - row.Date_received).days
    
    return 0

In [13]:
def extract_basic_info(dataset):
    dataset['Distance'] = dataset['Distance'].fillna(-1)
    dataset['Distance'] = dataset['Distance'] + 1

    dataset['Month_of_received'] = dataset.apply(lambda row: row.Date_received.month, axis=1)
    dataset['Day_of_received'] = dataset.apply(lambda row: row.Date_received.day, axis=1)
    dataset['Weekday_of_received'] = dataset.apply(lambda row: row.Date_received.weekday() + 1, axis=1)

    dataset['Base_consume'] = 0.0
    dataset['Discount'] = 0.0
    dataset['Discount_money'] = 0.0
    dataset['Coupon_type'] = 0
    dataset['Coupon_category'] = 0

    dataset = dataset.apply(lambda row: cal_coupon_feature(row), axis=1)

    dataset = dataset.sort_values(by=['User_id', 'Date_received'], ascending=True)

    dataset['Previous_user_id'] = dataset['User_id'].shift(1)
    dataset['Previous_date_received'] = dataset['Date_received'].shift(1)

    dataset['Next_user_id'] = dataset['User_id'].shift(-1)
    dataset['Next_date_received'] = dataset['Date_received'].shift(-1)

    dataset['Previous_duration'] = dataset.apply(lambda row: cal_previous_duration(row), axis=1)
    dataset['Next_duration'] = dataset.apply(lambda row: cal_next_duration(row), axis=1)
    return dataset

In [14]:
dataset = extract_basic_info(dataset)

In [15]:
dataset.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Month_of_received,Day_of_received,Weekday_of_received,Base_consume,Discount,Discount_money,Coupon_type,Coupon_category,Previous_user_id,Previous_date_received,Next_user_id,Next_date_received,Previous_duration,Next_duration
23866,209,5032,7557,20:5,2.0,2016-07-21,7,21,4,20.0,0.750000,5.0,1,9,NaN,NaT,209.0,2016-07-21,0,0
23867,209,5032,825,20:5,2.0,2016-07-21,7,21,4,20.0,0.750000,5.0,1,9,209.0,2016-07-21,215.0,2016-07-03,0,0
23793,215,599,5488,20:1,0.0,2016-07-03,7,3,7,20.0,0.950000,1.0,1,2,209.0,2016-07-21,316.0,2016-07-21,0,0
61826,316,2436,3992,30:5,1.0,2016-07-21,7,21,4,30.0,0.833333,5.0,1,4,215.0,2016-07-03,417.0,2016-07-12,0,0
23851,417,3507,12465,50:1,1.0,2016-07-12,7,12,2,50.0,0.980000,1.0,1,26,316.0,2016-07-21,432.0,2016-07-06,0,0


### 预测区间特征提取

In [16]:
d = dataset.copy()

+ **特征o1**: 用户在预测区获取的优惠券数量

In [17]:
t = d.groupby(['User_id']).size()
d = pd.merge(d, groupby2df(t, 'o1'), on=['User_id'], how='left')

+ **特征o2**: 用户平均15天领取的优惠券数量

In [18]:
d['o2'] = d['o1'] / 15

+ **特征o3**: 用户平均每天领取多少张优惠券

In [19]:
u = d[['User_id', 'o1']].drop_duplicates()

t = d.groupby('User_id')['Date_received'].max()
u = pd.merge(u, groupby2df(t, 'r_max'), on=['User_id'], how='left')

t = d.groupby('User_id')['Date_received'].min()
u = pd.merge(u, groupby2df(t, 'r_min'), on=['User_id'], how='left')

u['r_day_duration'] = u.apply(lambda row: (row['r_max'] - row['r_min']).days, axis=1)
u['o3'] = u['o1'] / u['r_day_duration']
d = pd.merge(d, u[['User_id', 'o3']], on=['User_id'], how='left')

+ **特征o4**:预测区用户每种类型优惠券领取的数量
+ **特征o5**:预测区用户每种类型优惠券领取的数量在所有领取的优惠券中的比率

In [20]:
t = d.groupby(['User_id', 'Coupon_category']).size()
d = pd.merge(d, groupby2df(t, 'o4'), on=['User_id', 'Coupon_category'], how='left')

In [21]:
d['o5'] = d['o4'] / d['o1']

+ **特征o6**:预测区用户领取优惠券Coupon_id领取的数量
+ **特征o8**:预测区用户领取优惠券Coupon_id领取的数量在所有领取的优惠券中的比率

In [22]:
t = d.groupby(['User_id', 'Coupon_id']).size()
d = pd.merge(d, groupby2df(t, 'o6'), on=['User_id', 'Coupon_id'], how='left')

In [23]:
d['o8'] = d['o6'] / d['o1']

+ **特征o7**:预测区用户领取优惠券Coupon_id在领取日领取的数量
+ **特征o9**:预测区用户领取优惠券Coupon_id在领取日领取的数量在所有领取的优惠券中的比率

In [24]:
t = d.groupby(['User_id', 'Coupon_id', 'Date_received']).size()
d = pd.merge(d, groupby2df(t, 'o7'), on=['User_id', 'Coupon_id', 'Date_received'], how='left')

In [25]:
d['o9'] = d['o7'] / d['o1']

+ **特征o10**:预测区用户领取多少种不同的优惠券
+ **特征o14**:预测区用户平均每个领取的优惠券领取了多少张
+ **特征o12**:预测区用户领取的不同的优惠券占所有优惠券的比率

In [26]:
t = d[['User_id', 'Coupon_id']].drop_duplicates()
t = t.groupby(['User_id']).size()
d = pd.merge(d, groupby2df(t, 'o10'), on=['User_id'], how='left')

In [27]:
d['o12'] = d['o10'] / d['Coupon_id'].unique().size

In [28]:
d['o14'] = d['o1'] / d['o10']

+ **特征o11**:预测区每种优惠券被领取的张数

In [29]:
t = dataset.groupby(['Coupon_id']).size()
d = pd.merge(d, groupby2df(t, 'o11'), on=['Coupon_id'], how='left')

+ **特征o13**:预测区用户领取的不同的商户数
+ **特征o16**:预测区用户领取的不同的商户数占所有商户数的比例

In [30]:
t = d[['User_id', 'Merchant_id']].drop_duplicates()
t = t.groupby(['User_id']).size()
d = pd.merge(d, groupby2df(t, 'o13'), on=['User_id'], how='left')

In [31]:
d['o16'] = d['o13'] / d['Merchant_id'].unique().size

+ **特征o15**:预测区用户在每个消费的商户领取的优惠券数
+ **特征o18**:预测区用户在每个消费的商户领取的优惠券数在所有领取的优惠券中的比率

In [32]:
t = dataset.groupby(['User_id', 'Merchant_id']).size()
d = pd.merge(d, groupby2df(t, 'o15'), on=['User_id', 'Merchant_id'], how='left')

In [33]:
d['o18'] = d['o15'] / d['o1']

+ **特征o19**:预测区用户在每个距离上领取的优惠券数量
+ **特征o20**:预测区用户在每个距离上领取的优惠券数量在所有领取的优惠券中的比率

In [34]:
t = dataset.groupby(['User_id', 'Distance']).size()
d = pd.merge(d, groupby2df(t, 'o19'), on=['User_id', 'Distance'], how='left')

In [35]:
d['o20'] = d['o19'] / d['o1']

+ **特征o21 - o23** 用户领取的优惠券距离最大、最小、平均

In [36]:
t = dataset.groupby(['User_id'])['Distance'].max()
d = pd.merge(d, groupby2df(t, 'o21'), on=['User_id'], how='left')

In [37]:
t = dataset.groupby(['User_id'])['Distance'].min()
d = pd.merge(d, groupby2df(t, 'o22'), on=['User_id'], how='left')

In [38]:
t = dataset.groupby(['User_id'])['Distance'].mean()
d = pd.merge(d, groupby2df(t, 'o23'), on=['User_id'], how='left')

+ **特征o17**:预测区用户领取的不同的优惠券分类的优惠券数量
+ **特征o24**:预测区用户领取的不同的优惠券分类的优惠券数量在所有领取的优惠券中的比率

In [39]:
t = dataset.groupby(['User_id', 'Coupon_type']).size()
d = pd.merge(d, groupby2df(t, 'o17'), on=['User_id', 'Coupon_type'], how='left')

In [40]:
d['o24'] = d['o17'] / d['o1']

+ **特征o25**:预测区用户在领取日领取的优惠券数量
+ **特征o26**:预测区用户在领取日领取的优惠券数量在所有领取的优惠券中的比率

In [41]:
t = d.groupby(['User_id', 'Date_received']).size()
d = pd.merge(d, groupby2df(t, 'o25'), on=['User_id', 'Date_received'], how='left')

In [42]:
d['o26'] = d['o25'] / d['o1']

+ **特征o27 - o29**用户优惠券折扣的最大、最小、平均值

In [43]:
t = d.groupby(['User_id'])['Discount'].mean()
d = pd.merge(d, groupby2df(t, 'o27'), on=['User_id'], how='left')

In [44]:
t = d.groupby(['User_id'])['Discount'].max()
d = pd.merge(d, groupby2df(t, 'o28'), on=['User_id'], how='left')

In [45]:
t = d.groupby(['User_id'])['Discount'].min()
d = pd.merge(d, groupby2df(t, 'o29'), on=['User_id'], how='left')

+ **特征o30**:预测区每个商户被领用的优惠券数量
+ **特征o38**:预测区每个商户平均被每个用户领取的数量

In [46]:
t = d.groupby(['Merchant_id']).size()
d = pd.merge(d, groupby2df(t, 'o30'), on=['Merchant_id'], how='left')

In [47]:
d['o38'] = d['o30'] / d['User_id'].unique().size

+ **特征o31**:预测区用户在每周不同的weekday领取优惠券的数量
+ **特征o39**:预测区用户在每周不同的weekday领取优惠券的数量在所有领取的优惠券中的比率
+ **特征o40**:预测区用户在每周不同的month领取优惠券的数量
+ **特征o41**:预测区用户在每周不同的month领取优惠券的数量在所有领取的优惠券中的比率
+ **特征o42**:预测区用户在每周不同的day领取优惠券的数量
+ **特征o43**:预测区用户在每周不同的day领取优惠券的数量在所有领取的优惠券中的比率

In [48]:
t = d.groupby(['User_id', 'Weekday_of_received']).size()
d = pd.merge(d, groupby2df(t, 'o31'), on=['User_id', 'Weekday_of_received'], how='left')

In [49]:
d['o39'] = d['o31'] / d['o1']

In [50]:
t = d.groupby(['User_id', 'Month_of_received']).size()
d = pd.merge(d, groupby2df(t, 'o40'), on=['User_id', 'Month_of_received'], how='left')

d['o41'] = d['o40'] / d['o1']

In [51]:
t = d.groupby(['User_id', 'Day_of_received']).size()
d = pd.merge(d, groupby2df(t, 'o42'), on=['User_id', 'Day_of_received'], how='left')

d['o43'] = d['o42'] / d['o1']

+ **特征o32**:预测区商户被多少不同用户领取
+ **特征o33**:预测区商户被不同用户平均领取优惠券数量

In [52]:
t = d[['Merchant_id', 'User_id']].drop_duplicates()
t = t.groupby(['Merchant_id']).size()
d = pd.merge(d, groupby2df(t, 'o32'), on=['Merchant_id'], how='left')

In [53]:
d['o33'] = d['o30'] / d['o32']

+ **特征o34**:每家商户有多少张不同的优惠券

In [54]:
t = d[['Merchant_id', 'Coupon_id']].drop_duplicates()
t = t.groupby(['Merchant_id']).size()
d = pd.merge(d, groupby2df(t, 'o34'), on=['Merchant_id'], how='left')

+ **特征o35**:每张优惠券被多少不同的人领取了
+ **特征o36**:每张优惠券平均被每个领用的用户领取了多少张
+ **特征o37**:每张优惠券平均被每个用户领取了多少张

In [55]:
t = d[['Coupon_id', 'User_id']].drop_duplicates()
t = t.groupby(['Coupon_id']).size()
d = pd.merge(d, groupby2df(t, 'o35'), on=['Coupon_id'], how='left')

In [56]:
d['o36'] = d['o11'] / d['o35']

In [57]:
d['o37'] = d['o11'] / d['User_id'].unique().size

+ **特征o44**:预测区用户next duration小于16天的次数

In [58]:
t = d[d['Next_duration']<16]
t = t.groupby(['User_id'])['Next_duration'].size()
d = pd.merge(d, groupby2df(t, 'o44'), on=['User_id'], how='left')

In [59]:
d = d.replace([np.inf, -np.inf], np.nan)

In [60]:
dataset = d.copy()

### Label抽取

In [61]:
if not IS_PRED:
    dataset['Duration'] = dataset.apply(lambda row: (row['Date'] - row['Date_received']).days, axis=1)
    dataset['Label'] = dataset.apply(lambda row: 1 if row['Duration'] < 16 else 0, axis=1)

In [62]:
dataset.head(10)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Month_of_received,Day_of_received,Weekday_of_received,Base_consume,...,o41,o42,o43,o32,o33,o34,o35,o36,o37,o44
0,209,5032,7557,20:5,2.0,2016-07-21,7,21,4,20.0,...,1.0,2,1.0,34,2.794118,5,25,1.000000,0.000328,2
1,209,5032,825,20:5,2.0,2016-07-21,7,21,4,20.0,...,1.0,2,1.0,34,2.794118,5,24,1.583333,0.000498,2
2,215,599,5488,20:1,0.0,2016-07-03,7,3,7,20.0,...,1.0,1,1.0,34,1.294118,2,32,1.312500,0.000550,1
3,316,2436,3992,30:5,1.0,2016-07-21,7,21,4,30.0,...,1.0,1,1.0,1970,1.050761,6,1650,1.000000,0.021623,1
4,417,3507,12465,50:1,1.0,2016-07-12,7,12,2,50.0,...,1.0,1,1.0,42,1.071429,1,42,1.071429,0.000590,1
5,432,2050,10438,200:20,3.0,2016-07-06,7,6,3,200.0,...,1.0,1,1.0,4472,1.074016,6,4212,1.000712,0.055236,1
6,448,760,13602,30:5,1.0,2016-07-01,7,1,5,30.0,...,1.0,1,0.2,29876,1.002745,6,29846,1.001307,0.391631,5
7,448,1482,10927,0.95,1.0,2016-07-04,7,4,1,0.0,...,1.0,1,0.2,1,4.000000,1,1,4.000000,0.000052,5
8,448,1482,10927,0.95,1.0,2016-07-06,7,6,3,0.0,...,1.0,1,0.2,1,4.000000,1,1,4.000000,0.000052,5
9,448,1482,10927,0.95,1.0,2016-07-07,7,7,4,0.0,...,1.0,1,0.2,1,4.000000,1,1,4.000000,0.000052,5


## 用户特征抽取

In [63]:
def get_count(df, keys, suffix):
    if 'Coupon_id' not in keys and 'Discount_rate' not in keys:
        t = full.groupby(keys).size()
        df = pd.merge(df, groupby2df(t, suffix+'0'), on=keys, how='left')
    
    t = receive.groupby(keys).size()
    df = pd.merge(df, groupby2df(t, suffix+'1'), on=keys, how='left')

    t = consume.groupby(keys).size()
    df = pd.merge(df, groupby2df(t, suffix+'2'), on=keys, how='left')

    t = use.groupby(keys).size()
    df = pd.merge(df, groupby2df(t, suffix+'3'), on=keys, how='left')

    df[suffix+'4'] = df[suffix+'1'] - df[suffix+'3']
    df[suffix+'5'] = df[suffix+'1'] - df[suffix+'2']
    
    df[suffix+'6'] = df[suffix+'3'] / df[suffix+'4']
    df[suffix+'7'] = df[suffix+'2'] / df[suffix+'5']
    return df

In [64]:
def process_mean_max_min(d, df, keys, column_name, prefix, count):
    t = d.groupby(keys)[column_name].mean()
    df = pd.merge(df, groupby2df(t, prefix+str(count)), on=keys, how='left')

    count = count + 1
    t = d.groupby(keys)[column_name].max()
    df = pd.merge(df, groupby2df(t, prefix+str(count)), on=keys, how='left')

    count = count + 1
    t = d.groupby(keys)[column_name].min()
    df = pd.merge(df, groupby2df(t, prefix+str(count)), on=keys, how='left')
    return df

In [65]:
def get_unique_size(d, df, groups, keys, column_name):
    t = d[groups].drop_duplicates()
    t = t.groupby(keys).size()
    df = pd.merge(df, groupby2df(t, column_name), on=keys, how='left')
    return df

### 拆分特征数据集合

In [66]:
feature_alpha_offline = extract_basic_info(feature_alpha_offline)

feature_alpha_offline['Duration'] = feature_alpha_offline.apply(lambda row: (row['Date'] - row['Date_received']).days, axis=1)
feature_alpha_offline['Label'] = feature_alpha_offline.apply(lambda row: 1 if row['Duration'] < 16 else 0, axis=1)

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying 

In [67]:
full = feature_alpha_offline[feature_alpha_offline.Coupon_id.isnull()].copy()
receive = feature_alpha_offline[feature_alpha_offline.Coupon_id>0].copy()
consume = receive[receive.Duration >= 0].copy()
use = receive[receive.Label==1].copy()

### 用户特征

In [68]:
u = feature_alpha_offline[feature_alpha_offline['Coupon_id']>0][['User_id']].drop_duplicates()

+ **特征u1**:用户领取优惠券的数量
+ **特征u2**:用户消费优惠券的数量
+ **特征u3**:用户15天内消费优惠券的数量
+ **特征u4**:用户15天内没有消费优惠券的数量
+ **特征u5**:用户用户没有消费优惠券的数量
+ **特征u6**:用户15天内消费优惠券的数量 比 用户15天内没有消费优惠券的数量
+ **特征u7**:用户消费优惠券的数量 比 用户用户没有消费优惠券的数量

In [69]:
u = get_count(u, ['User_id'], 'u')

+ **特征u8**:用户消费优惠券消费天数的平均值
+ **特征u9**:用户消费优惠券消费天数的最大值
+ **特征u10**:用户消费优惠券消费天数的最小值

In [70]:
u = process_mean_max_min(consume, u, ['User_id'], 'Duration', 'u', 8)

+ **特征u11**:用户消费优惠券折扣率的平均值
+ **特征u12**:用户消费优惠券折扣率的最大值
+ **特征u13**:用户消费优惠券折扣率的最小值

In [71]:
u = process_mean_max_min(consume, u, ['User_id'], 'Discount', 'u', 11)

+ **特征u14**:用户消费多少种不同优惠券
+ **特征u15**:用户15天内消费多少种不同优惠券

In [72]:
u = get_unique_size(consume, u, ['User_id', 'Coupon_id'], ['User_id'], 'u14')
u = get_unique_size(use, u, ['User_id', 'Coupon_id'], ['User_id'], 'u15')

+ **特征u16**:用户消费多少个不同商家的优惠券
+ **特征u17**:用户15天内消费多少个不同商家的优惠券

In [73]:
u = get_unique_size(consume, u, ['User_id', 'Merchant_id'], ['User_id'], 'u16')
u = get_unique_size(use, u, ['User_id', 'Merchant_id'], ['User_id'], 'u17')

+ **特征u18**:用户15天内消费优惠券的核销率
+ **特征u19**:用户消费优惠券的核销率
+ **特征u20**:用户15天内平均消费领取天数在15天内的优惠券的数量
+ **特征u21**:用户15天内平均消费优惠券的数量
+ **特征u22**:用户15天内平均领取优惠券的数量

In [74]:
u['u18'] = u['u3'] / u['u1']
u['u19'] = u['u2'] / u['u1']

u['u20'] = u['u3'] / 15
u['u21'] = u['u2'] / 15
u['u22'] = u['u1'] / 15

In [75]:
u = process_mean_max_min(use, u, ['User_id'], 'Duration', 'u', 23)

In [76]:
dataset = pd.merge(dataset, u, on=['User_id'], how='left')

### 用户-优惠券分类特征

In [77]:
ucc = feature_alpha_offline[feature_alpha_offline['Coupon_id']>0][['User_id', 'Coupon_category']].drop_duplicates()

In [78]:
ucc = get_count(ucc, ['User_id', 'Coupon_category'], 'ucc')

In [79]:
ucc = process_mean_max_min(consume, ucc, ['User_id', 'Coupon_category'], 'Duration', 'ucc', 8)

In [80]:
dataset = pd.merge(dataset, ucc, on=['User_id', 'Coupon_category'], how='left')

In [81]:
dataset['ucc11'] = dataset['ucc3'] / dataset['u1']
dataset['ucc12'] = dataset['ucc2'] / dataset['u1']

### 用户 - 优惠券特征

In [82]:
uc = feature_alpha_offline[feature_alpha_offline['Coupon_id']>0][['User_id', 'Coupon_id']].drop_duplicates()

+ **特征uc1**:不同coupon_id,用户领取优惠券的数量
+ **特征uc2**:不同coupon_id,用户消费优惠券的数量
+ **特征uc3**:不同coupon_id,用户15天内消费优惠券的数量
+ **特征uc4**:不同coupon_id,用户15天内没有消费优惠券的数量
+ **特征uc5**:不同coupon_id,用户用户没有消费优惠券的数量
+ **特征uc6**:不同coupon_id,用户15天内消费优惠券的数量 比 用户15天内没有消费优惠券的数量
+ **特征uc7**:不同coupon_id,用户消费优惠券的数量 比 用户用户没有消费优惠券的数量

In [83]:
uc = get_count(uc, ['User_id', 'Coupon_id'], 'uc')

+ **特征uc8**:不同coupon_id,用户消费优惠券消费天数的平均值
+ **特征uc9**:不同coupon_id,用户消费优惠券消费天数的最大值
+ **特征uc10**:不同coupon_id,用户消费优惠券消费天数的最小值

In [84]:
uc = process_mean_max_min(consume, uc, ['User_id', 'Coupon_id'], 'Duration', 'uc', 8)

In [85]:
dataset = pd.merge(dataset, uc, on=['User_id', 'Coupon_id'], how='left')

+ **特征uc11**:不同coupon_id,用户15天内消费优惠券的核销率
+ **特征uc12**:不同coupon_id,用户消费优惠券的核销率

In [86]:
dataset['uc11'] = dataset['uc3'] / dataset['u1']
dataset['uc12'] = dataset['uc2'] / dataset['u1']

### 用户 - 距离特征

In [87]:
ud = feature_alpha_offline[feature_alpha_offline['Coupon_id']>0][['User_id', 'Distance']].drop_duplicates()

In [88]:
ud = get_count(ud, ['User_id', 'Distance'], 'ud')

In [89]:
ud = process_mean_max_min(consume, ud, ['User_id', 'Distance'], 'Duration', 'ud', 8)

In [90]:
dataset = pd.merge(dataset, ud, on=['User_id', 'Distance'], how='left')

In [91]:
dataset['ud11'] = dataset['ud3'] / dataset['u1']
dataset['ud12'] = dataset['ud2'] / dataset['u1']

### 用户 - 商户特征

In [92]:
um = feature_alpha_offline[feature_alpha_offline['Coupon_id']>0][['User_id', 'Merchant_id']].drop_duplicates()

+ **特征um1**:不同merchant_id,用户领取优惠券的数量
+ **特征um2**:不同merchant_id,用户消费优惠券的数量
+ **特征um3**:不同merchant_id,用户15天内消费优惠券的数量
+ **特征um4**:不同merchant_id,用户15天内没有消费优惠券的数量
+ **特征um5**:不同merchant_id,用户用户没有消费优惠券的数量
+ **特征um6**:不同merchant_id,用户15天内消费优惠券的数量 比 用户15天内没有消费优惠券的数量
+ **特征um7**:不同merchant_id,用户消费优惠券的数量 比 用户用户没有消费优惠券的数量

In [93]:
um = get_count(um, ['User_id', 'Merchant_id'], 'um')

+ **特征um8**:不同merchant_id,用户消费优惠券消费天数的平均值
+ **特征um9**:不同merchant_id,用户消费优惠券消费天数的最大值
+ **特征um10**:不同merchant_id,用户消费优惠券消费天数的最小值

In [94]:
um = process_mean_max_min(consume, um, ['User_id', 'Merchant_id'], 'Duration', 'um', 8)

In [95]:
t = feature_alpha_offline[feature_alpha_offline.Coupon_id == feature_alpha_offline.Coupon_id]
t = t.groupby(['User_id', 'Merchant_id']).size().reset_index()
t = t.groupby('User_id').size()
um = pd.merge(um, groupby2df(t, 'um16'), on=['User_id'], how='left')

In [96]:
um = get_unique_size(feature_alpha_offline, um, ['User_id', 'Merchant_id'], ['User_id'], 'um15')

In [97]:
um['um17']= um.um15 / um.um16

In [98]:
dataset = pd.merge(dataset, um, on=['User_id', 'Merchant_id'], how='left')

+ **特征um11**:不同merchant_id,用户15天内消费优惠券的核销率
+ **特征um12**:不同merchant_id,用户消费优惠券的核销率

In [99]:
dataset['um11'] = dataset['um3'] / dataset['u1']
dataset['um12'] = dataset['um2'] / dataset['u1']

+ **特征um13**:不同merchant_id,用户15天内消费优惠券在15天内平均核销数
+ **特征um14**:不同merchant_id,用户消费优惠券在15天内平均核销数

In [100]:
dataset['um13'] = dataset['um3'] / 15
dataset['um14'] = dataset['um2'] / 15

### 商户特征

In [101]:
m = feature_alpha_offline[feature_alpha_offline['Coupon_id']>0][['Merchant_id']].drop_duplicates()

In [102]:
m = get_count(m, ['Merchant_id'], 'm')

In [103]:
m = process_mean_max_min(consume, m, ['Merchant_id'], 'Duration', 'm', 8)

In [104]:
m = get_unique_size(consume, m, ['Merchant_id', 'User_id'], ['Merchant_id'], 'm11')
m = get_unique_size(use, m, ['Merchant_id', 'User_id'], ['Merchant_id'], 'm12')

In [105]:
m['m13'] = m['m2'] / m['m11']
m['m14'] = m['m3'] / m['m12']

In [106]:
dataset = pd.merge(dataset, m, on=['Merchant_id'], how='left')

### 优惠券特征

In [107]:
c = feature_alpha_offline[feature_alpha_offline['Coupon_id']>0][['Coupon_id']].drop_duplicates()

In [108]:
c = get_count(c, ['Coupon_id'], 'c')

In [109]:
c = process_mean_max_min(consume, c, ['Coupon_id'], 'Duration', 'c', 8)

In [110]:
c = get_unique_size(consume, c, ['Coupon_id', 'User_id'], ['Coupon_id'], 'c11')
c = get_unique_size(use, c, ['Coupon_id', 'User_id'], ['Coupon_id'], 'c12')

In [111]:
c['c13'] = c['c2'] / c['c11']
c['c14'] = c['c3'] / c['c12']

In [112]:
dataset = pd.merge(dataset, c, on=['Coupon_id'], how='left')

In [113]:
cd = feature_alpha_offline[feature_alpha_offline['Coupon_id']>0][['Coupon_id', 'Date_received']].drop_duplicates()

In [114]:
cd = get_count(cd, ['Coupon_id', 'Date_received'], 'cd')

In [115]:
dataset = pd.merge(dataset, cd, on=['Coupon_id', 'Date_received'], how='left')

### 优惠券分类特征

In [116]:
dr = feature_alpha_offline[feature_alpha_offline['Coupon_id']>0][['Discount_rate']].drop_duplicates()

In [117]:
dr = get_count(dr, ['Discount_rate'], 'dr')

In [118]:
dataset = pd.merge(dataset, dr, on=['Discount_rate'], how='left')

### 线上用户特征

In [119]:
ou = feature_alpha_online[['User_id']].drop_duplicates()

t = feature_alpha_online.groupby(['User_id']).size()
ou = pd.merge(ou, groupby2df(t, 'ou1'), on=['User_id'], how='left')

t = feature_alpha_online[feature_alpha_online.Action == 0].groupby(['User_id']).size()
ou = pd.merge(ou, groupby2df(t, 'ou2'), on=['User_id'], how='left')

t = feature_alpha_online[feature_alpha_online.Action == 1].groupby(['User_id']).size()
ou = pd.merge(ou, groupby2df(t, 'ou3'), on=['User_id'], how='left')

t = feature_alpha_online[feature_alpha_online.Action == 2].groupby(['User_id']).size()
ou = pd.merge(ou, groupby2df(t, 'ou4'), on=['User_id'], how='left')

In [120]:
dataset = pd.merge(dataset, ou, on=['User_id'], how='left')

## 构造特征选择器

In [121]:
len(dataset.columns.values)

193

In [128]:
dataset.columns.values

array(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Month_of_received', 'Day_of_received',
       'Weekday_of_received', 'Base_consume', 'Discount',
       'Discount_money', 'Coupon_type', 'Coupon_category',
       'Previous_user_id', 'Previous_date_received', 'Next_user_id',
       'Next_date_received', 'Previous_duration', 'Next_duration', 'o1',
       'o2', 'o3', 'o4', 'o5', 'o6', 'o8', 'o7', 'o9', 'o10', 'o12',
       'o14', 'o11', 'o13', 'o16', 'o15', 'o18', 'o19', 'o20', 'o21',
       'o22', 'o23', 'o17', 'o24', 'o25', 'o26', 'o27', 'o28', 'o29',
       'o30', 'o38', 'o31', 'o39', 'o40', 'o41', 'o42', 'o43', 'o32',
       'o33', 'o34', 'o35', 'o36', 'o37', 'o44', 'u0', 'u1', 'u2', 'u3',
       'u4', 'u5', 'u6', 'u7', 'u8', 'u9', 'u10', 'u11', 'u12', 'u13',
       'u14', 'u15', 'u16', 'u17', 'u18', 'u19', 'u20', 'u21', 'u22',
       'u23', 'u24', 'u25', 'ucc0', 'ucc1', 'ucc2', 'ucc3', 'ucc4',
       'ucc5', 'ucc6', 'ucc7', 'ucc8', 'u

In [123]:
dataset = dataset.replace([np.inf, -np.inf], np.nan)

In [124]:
continous = [
    'Distance','Coupon_id','Coupon_type', 'Coupon_category',
    'Month_of_received', 'Day_of_received',
       'Weekday_of_received', 'Base_consume', 'Discount',
       'Discount_money', 
    'Previous_duration', 'Next_duration', 'o1',
       'o2', 'o3', 'o4', 'o5', 'o6', 'o8', 'o7', 'o9', 'o10', 'o12',
       'o14', 'o11', 'o13', 'o16', 'o15', 'o18', 'o19', 'o20', 'o21',
       'o22', 'o23', 'o17', 'o24', 'o25', 'o26', 'o27', 'o28', 'o29',
       'o30', 'o38', 'o31', 'o39', 'o40', 'o41', 'o42', 'o43', 'o32',
       'o33', 'o34', 'o35', 'o36', 'o37', 'o44', 
       'u0', 'u1', 'u2', 'u3', 'u4', 'u5', 'u6', 'u7', 'u8', 'u9', 'u10',
       'u11', 'u12', 'u13', 'u14', 'u15', 'u16', 'u17', 'u18', 'u19',
       'u20', 'u21', 'u22', 'u23', 'u24', 'u25', 'ucc0', 'ucc1', 'ucc2',
       'ucc3', 'ucc4', 'ucc5', 'ucc6', 'ucc7', 'ucc8', 'ucc9', 'ucc10',
       'ucc11', 'ucc12', 'uc1', 'uc2', 'uc3', 'uc4', 'uc5', 'uc6', 'uc7',
       'uc8', 'uc9', 'uc10', 'uc11', 'uc12', 'ud0', 'ud1', 'ud2', 'ud3',
       'ud4', 'ud5', 'ud6', 'ud7', 'ud8', 'ud9', 'ud10', 'ud11', 'ud12',
       'um0', 'um1', 'um2', 'um3', 'um4', 'um5', 'um6', 'um7', 'um8',
       'um9', 'um10', 'um16', 'um15', 'um17', 'um11', 'um12', 'um13', 'um14', 'm0',
       'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11',
       'm12', 'm13', 'm14', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7',
       'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'cd1', 'cd2', 'cd3',
       'cd4', 'cd5', 'cd6', 'cd7', 'dr1', 'dr2', 'dr3', 'dr4', 'dr5',
       'dr6', 'dr7', 'ou1', 'ou2', 'ou3', 'ou4']

label = ['Label']

In [125]:
if not IS_PRED:
    feature_processor = Pipeline([
        ('features', FeatureUnion([
            ('continuous', Pipeline([
                ('extract', ColumnSelector(continous)),
                ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    #             ('scale', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('normalize', Normalizer())
            ])),
        ])),
    #     ('sc4gbdt', StandardScaler()),
    ])

    feature_processor.fit(dataset, dataset['Label'].values.ravel())
    selector_model = XGBClassifier(max_depth=3, n_estimators=20, random_state=0)
    selector_model.fit(feature_processor.transform(dataset), dataset['Label'].values.ravel())

    feature_selector = []
    for index, value in enumerate(selector_model.feature_importances_):
        if value > 0:
            feature_selector.append((continous[index], value))

    feature_selector

## 保存数据

In [126]:
dataset = dataset.replace([np.inf, -np.inf], np.nan)

In [127]:
dataset.to_csv('../features/' + FILENAME + '.csv')